# Introduction to vector databases on S&P 500 news

 # 📌 Objectives

 By the end of this notebook, students will be able to:

 1. **Load and Explore Financial News Data:**
    - Load a preprocessed dataset of financial news headlines and summaries related to S&P 500 companies.
    - Inspect and structure relevant metadata including publication date, ticker, and provider.

 2. **Build a Custom Vector Store:**
    - Combine title and summary text for embedding.
    - Implement a vector store from scratch with support for semantic search and optional metadata-based filtering using cosine similarity.

 3. **Generate and Use Sentence Embeddings:**
    - Apply a pre-trained transformer model (all-MiniLM-L6-v2) to encode text into embeddings for semantic analysis.

 4. **Perform Semantic Search with Filtering:**
    - Retrieve the most relevant news documents for a given query using both full-dataset and metadata-filtered searches.

 5. **Analyze Semantic Search Results:**
    - Interpret and visualize results of search queries (e.g., "AI announcement").
    - Identify temporal patterns and ticker frequency among top-ranked results.

 6. **Implement a FAISS-Based Vector Store:**
    - Normalize embeddings and use FAISS (IndexFlatIP) to enable efficient similarity search at scale.
    - Compare FAISS-based search results with custom vector store results.

 7. **Critically Compare Search Systems:**
    - Evaluate differences in similarity computation, speed, and scalability between custom and FAISS-based systems.
    - Reflect on design trade-offs for real-world vector search systems.

## Install and Import important libraries

In [ ]:
# %pip install sentence-transformers
# %pip install faiss-cpu

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss
import yfinance as yf

## Load news data

Load the provided news dataset and inspect the contents.

 👉 **Instructions**:
 - Load the CSV file named df_news.csv into a pandas DataFrame.
 - Convert the PUBLICATION_DATE column to a proper date format (not datetime).
 - Display the first few rows to understand the data structure. The key columns you’ll be using are:
   - TITLE
   - SUMMARY
   - TICKER
   - PROVIDER
   - PUBLICATION_DATE


In [ ]:
df_news = pd.read_csv('./data/df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

## Implement custom vector store

You will now implement a basic vector store from scratch. This class will allow you to:
1. Store embedded text and metadata.
2. Perform filtered semantic search using cosine similarity.

 👉 **Instructions**:

### Step 1: Prepare Documents
 - Combine the TITLE and SUMMARY columns into a single column named EMBEDDED_TEXT.
 - This combined text will be embedded later.

### Step 2: Load Embedding Model
 - Load the 'all-MiniLM-L6-v2' model from sentence-transformers.

### Step 3: Define CustomVectorStore Class
 Implement the class with the following methods (functions):

 - '__init__': Accepts an embedding model, list of documents, and corresponding metadata. Computes and stores embeddings for all documents.
 - search: Takes a query and returns the top k most similar documents, optionally filtered using metadata.
   - If a metadata_filter function is provided, apply it before computing similarity.
   - Use cosine similarity to compute distances.
   - Return the top k results as tuples of: (document, metadata, similarity score).

 ✅ **Requirements**:
 - Ensure the number of documents matches the number of metadata entries.
 - Make use of sklearn.metrics.pairwise.cosine_similarity.

You will test your class at the end.

In [ ]:
# Create the EMBEDDED_TEXT column

df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' ' + df_news['SUMMARY']

display(df_news.head())

In [ ]:
# Load the embedding model

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
class CustomVectorStore:
    def __init__(self,
                 embedding_model: SentenceTransformer,
                 documents: list[str],
                 metadata: list[dict[str, str]]):
        self.embedding_model = embedding_model
        self.documents = documents
        self.metadata = metadata

        if len(documents) != len(metadata):
            raise ValueError("Number of documents must match number of metadata entries.")

        self.embeddings = self.embedding_model.encode(documents)

    def search(self,
               query: str,
               k: int = 5,
               metadata_filter: dict[str, str] = None):

        mask = self._filter_to_mask(metadata_filter)

        similarities = cosine_similarity(
            self.embedding_model.encode([query]),
            np.array([x for x in self._masked(self.embeddings, mask)])
        )[0]

        result = (
            (d, m, s) for d, m, s in zip(
                self._masked(self.documents, mask),
                self._masked(self.metadata, mask),
                similarities
            )
        )

        # Sort by similarity (third item) desc, return top k results
        return sorted(result, key = lambda x: x[2], reverse=True)[:k]

    def _filter_to_mask(self, metadata_filter):
        if metadata_filter is None:
            return [True for _ in self.metadata]

        return [
            self._is_metadata_match(self.metadata[i], metadata_filter) for i in range(len(self.metadata))
        ]

    @staticmethod
    def _is_metadata_match(document_metadata, metadata_filter):
        for key, value in metadata_filter.items():
            if key not in document_metadata or document_metadata[key] != value:
                return False
        return True

    @staticmethod
    def _masked(items, mask):
        return (item for i, item in enumerate(items) if mask[i])



## Create and populate the vector store

 👉 **Instructions**:
 - Convert the combined EMBEDDED_TEXT column into a list of strings.
 - Create a metadata list of dictionaries for each document with keys:
   - 'PUBLICATION_DATE'
   - 'TICKER'
   - 'PROVIDER'
 - Instantiate your CustomVectorStore using:
   - The embedding model.
   - The list of document strings.
   - The metadata list.

 After initializing the store, you should be able to perform searches using:
'store.search("Apple earnings", k=3)'


In [ ]:
documents = df_news['EMBEDDED_TEXT'].tolist()

display(documents[:5])  # Display first 5 documents to verify

In [ ]:
metadata = []

for row in df_news.itertuples():
    metadata.append({
        'PUBLICATION_DATE': row.PUBLICATION_DATE,
        'TICKER': row.TICKER,
        'PROVIDER': row.PROVIDER
    })

display(metadata[:5])  # Display first 5 metadata entries to verify

In [ ]:
vector_store = CustomVectorStore(
    embedding_model=embedding_model,
    documents=documents,
    metadata=metadata
)

In [ ]:
vector_store.search("Apple earnings", k=3)  # Test the search method

## Using your vector store and analyzing the results


### Retrieve AI Announcements

 Now that your vector store is working, let's use it to retrieve relevant documents.

 👉 **Instructions**:
 - Perform a semantic search using the query `"AI announcement"` with `k=50` to retrieve the top 50 most relevant articles.
 - Display each result's:
   - `TICKER`
   - `PUBLICATION_DATE`
   - `PROVIDER`
   - Similarity score
   - Full document text


In [ ]:
def display_search_results(results):
    for doc, meta, similarity in results:
        print("="*64)
        print(f"{meta['TICKER']:<6} | {meta['PUBLICATION_DATE']} | {meta['PROVIDER']}")
        print(f"Similarity: {similarity:.4f}")
        print("-"*64)
        print(doc)
        print("="*64)
        print("\n\n")

In [ ]:
results = vector_store.search("AI announcement", k=50)

display_search_results(results)

 ### **Q1.** What are the top companies (by TICKER) most frequently appearing in the top 50 search results for the query "AI announcement"?
 Check their company names on Yahoo Finance. Are you surprised by the results?

 ✅ **Your task**:
 - Count the number of times each TICKER appears.
 - Print and analyze the top results.

In [ ]:
count_by_ticker = {}

for _, meta, _ in results:
    t = meta['TICKER']

    if t in count_by_ticker:
        count_by_ticker[t] += 1

    else:
        count_by_ticker[t] = 1

sorted_tickers_count = sorted(count_by_ticker.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# Focusing analysis on tickers appearing more than once
filtered_tickers = [(t, c) for t, c in sorted_tickers_count if c > 1]

In [ ]:
for t, c in filtered_tickers:
    comp_name = yf.Ticker(t).info['longName']
    print(f"{comp_name} [{t}] appeared in {c} articles about 'AI announcement'")

Announcements from chipmakers (Intel, AMD, Qualcomm) are to be expected since they are developing new chips with AI accelerators.

Microsoft and Amazon were also expected, since they provide the cloud computing services to execute AI workloads, and both of them are introducing AI heavily on their core products. Meta is a similar story; while they don't provide cloud services, they are integrating AI in all their applications, and they are also heavily invested in the development of LLMs.

Palantir is the company with the most appearances, and while I may not be that familiar with the company, I do know that they operate in the military sector, which involves a lot of government contracts as well.

Baker Hughes and Expedia may not be the first companies the general population thinks about when talking about AI; however, it makes sense to see them in the list since they are starting to adopt AI technologies.

Lastly, Cognizant is a consulting company, and that industry is currently investing very heavily in implementing AI solutions for partners and customers.


### **Q2.** What is the date range of the top 50 results? Are the articles evenly distributed across time?

 ✅ **Your task**:
 - Extract the publication dates of the top 50 results.
 - Plot a histogram to visualize the temporal distribution.
 - Reflect: Are these mentions clustered in recent months or spread evenly?

 🧠 **Follow-up**: What could explain the timing of increased AI announcements?

In [ ]:
count_by_date = {}

for _, meta, _ in results:
    _date = meta['PUBLICATION_DATE']

    if _date in count_by_date:
        count_by_date[_date] += 1

    else:
        count_by_date[_date] = 1

In [ ]:
print(f"Date ranges from {min(count_by_date.keys())} to {max(count_by_date.keys())}")

In [ ]:
plt.bar(count_by_date.keys(), count_by_date.values())
plt.xlabel('Publication date')
plt.ylabel('Number of articles')
plt.show()

Articles about AI announcements are not evenly distributed across time. In fact, they are mostly clustered in the second half of May, 2025.

## Implement FAISS vector store

 We’ll now implement a **FAISS-based vector store**, designed for efficient similarity search at scale.

 👉 **Steps**:
 - Use the same sentence transformer model: all-MiniLM-L6-v2.
 - Encode your text into embessings
 - Normalize the vectors.

 FAISS supports multiple similarity types. For cosine similarity, we must:

 ✅ Normalize vectors to unit length
 ✅ Use `IndexFlatIP` (inner product), because:
 - For normalized vectors, inner product = cosine similarity


In [ ]:
embeddings = embedding_model.encode(documents)

In [ ]:
faiss.normalize_L2(embeddings)

In [ ]:
embedding_size = embeddings.shape[1]

index = faiss.IndexFlatIP(embedding_size)

We’ll now:
 - Initialize the FAISS index
 - Add the document embeddings

In [ ]:
index.add(embeddings)

In [ ]:
q = embedding_model.encode(["AI Announcement"])
faiss.normalize_L2(q)
d, i = index.search(q, k=5)

### Create a `FaissVectorStore` class with a `.search()` method.

 This method:
 - Encodes and normalizes the query
 - Optionally applies metadata filters
 - Returns top `k` results with similarity scores and metadata

In [ ]:
class FaissVectorStore:
    def __init__(self,
                 embedding_model: SentenceTransformer,
                 documents: list[str],
                 metadata: list[dict[str, str]],
                 index: faiss.IndexFlatIP):
        self.embedding_model = embedding_model
        self.documents = documents
        self.metadata = metadata
        self.index = index

    def search(self,
               query: str,
               k: int = 5,
               metadata_filter: dict[str, str] = None):

        query_embedding = self.embedding_model.encode([query])
        faiss.normalize_L2(query_embedding)

        if metadata_filter is None:
            distances, indexes = self.index.search(query_embedding, k)
            return self._to_search_result(distances, indexes)

        else:
            # First create a SearchParameters object that excludes any document
            # that doesn't match the metadata.
            search_ids = self._filter_documents_id(metadata_filter)
            idx_id_selector = faiss.IDSelectorBatch(search_ids)
            idx_search_params = faiss.SearchParameters(sel=idx_id_selector)

            distances, indexes = self.index.search(
                query_embedding, k, params=idx_search_params)

            return self._to_search_result(distances, indexes)

    def _to_search_result(self, distances, indexes):
        r = []

        # distances and indexes are of shape (n_queries, n_results)
        # Because this class only supports one query at a time,
        # we can safely get the first row of the arrays
        for d, i in zip(distances[0], indexes[0]):
            r.append((self.documents[i], self.metadata[i], d))
        return r

    def _filter_documents_id(self, metadata_filter):
        indexes = []
        for i in range(len(self.metadata)):
            if self._is_metadata_match(self.metadata[i], metadata_filter):
                indexes.append(i)
        return indexes

    @staticmethod
    def _is_metadata_match(document_metadata, metadata_filter):
        for key, value in metadata_filter.items():
            if key not in document_metadata or document_metadata[key] != value:
                return False
        return True

 Let’s run a semantic search using FAISS:
 - Query: `"AI announcement"`
 - Top results: `k=50`

 Print out:
 - Ticker
 - Publication date
 - Provider
 - Cosine similarity score
 - Full text

In [ ]:
faiss_vector_store = FaissVectorStore(
    embedding_model=embedding_model,
    documents=documents,
    metadata=metadata,
    index=index
)

In [ ]:
faiss_results = faiss_vector_store.search("AI announcement", k=50)
display_search_results(faiss_results)

## Evaluation: Comparing FAISS Vector Store with your Custom Vector Store

### Retrieval Comparison Across Systems

 👉 **Instructions**:

 - Using the 5 'test_queries' provided in the list below, retrieve the top 5 news for each query using both your custom and FAISS vector stores.
 - Inspect how similar the returned news texts are for each query.
 - Focus especially on the order of results (not just their presence).


In [ ]:
test_queries = [
    'Stock price drop',
    'Layoffs',
    'Mergers and acquisitions',
    'Fed interest rate',
    'Regulation',
    'Cryptocurrency'

]

In [ ]:
def compare_search_results(a, b, labels=['Results 1', 'Results 2']):
    for a_i, b_i in zip(a,b):
        doc_a, meta_a, similarity_a = a_i
        doc_b, meta_b, similarity_b = b_i

        lines = max(len(doc_a), len(doc_b))//60 + 1

        print(f"{labels[0]:=^64}\t{labels[1]:=^64}")

        a_meta = f"{meta_a['TICKER']:<6} | {meta_a['PUBLICATION_DATE']} | {meta_a['PROVIDER']}"
        b_meta = f"{meta_b['TICKER']:<6} | {meta_b['PUBLICATION_DATE']} | {meta_b['PROVIDER']}"
        print(f"{a_meta:<64}\t{b_meta}")

        print(f"{f'Similarity: {similarity_a:.4f}':<64}\t{f'Similarity: {similarity_b:.4f}':<64}")

        print(f"{'-'*64}\t{'-'*64}")
        for i in range(lines):
            print(f"{doc_a[i*60:(i+1)*60]:<64}\t{doc_b[i*60:(i+1)*60]:<64}")

        print(f"{'='*64}\t{'='*64}")
        print("\n\n")

In [ ]:
for tq in test_queries:
    print(f"Query: '{tq}'")

    custom_result = vector_store.search(tq, k=5)
    faiss_result = faiss_vector_store.search(tq, k=5)

    compare_search_results(custom_result, faiss_result, labels=['Custom Vector Store', 'FAISS'])

### **Q3.** Do you observe any significant differences in the top-5 retrieved results for each query between the two systems? Explain why these differences may or may not occur.

The results are always the same, in terms of the documents returned, their similarity score and ranking. This is because we are using the exact same embedding model and the same similarity measure (cosine similarity). If we were to change any of these parameters (for example, by using a diferent type of faiss index), we might get different results.

## Evaluation: Comparing 2 embedding models with your Custom Vector Store

 👉 **Instructions**:

 - Implement your custom vector store with `'all-MiniLM-L6-v2'` model from `sentence-transformers`, and compare it with the `'all-mpnet-base-v2'` model from `sentence-transformers`.
 - Similarly, retrieve the top 5 news for each query using both implementations.
 - Inspect how similar the returned news texts are for each query.
 - Focus especially on the order of results (not just their presence).

In [ ]:
all_minilm_model = SentenceTransformer('all-MiniLM-L6-v2')
all_mpnet_model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
mini_lm_vector_store = CustomVectorStore(
    embedding_model=all_minilm_model,
    documents=documents,
    metadata=metadata)

In [ ]:
mpnet_vector_store = CustomVectorStore(
    embedding_model=all_mpnet_model,
    documents=documents,
    metadata=metadata)

In [ ]:
for tq in test_queries:
    print(f"Query: '{tq}'")

    a_result = mini_lm_vector_store.search(tq, k=5)
    b_result = mpnet_vector_store.search(tq, k=5)

### **Q4.**  Which embedding model performs better in retrieving relevant documents for the queries? Support your answer with specific examples from the results.

 YOUR WRITTEN RESPONSE HERE

## 🛠️ Implementation-Level Questions

### **Q5.**  What role does vector normalization play in each implementation?


 YOUR WRITTEN RESPONSE HERE



### **Q6.**  What would happen if vectors were not normalized before indexing?


 YOUR WRITTEN RESPONSE HERE




## 🧠 Critical Thinking / Design Choices


### **Q7.** Which implementation would you choose for a production-scale search system? Why?



 YOUR WRITTEN RESPONSE HERE




### **Q8.**   If your dataset grows from 5K to 5M articles, which implementation will scale better? Justify your answer.


 YOUR WRITTEN RESPONSE HERE
